In [1]:
import numpy as np
import pandas as pd

n = 480
rack_letters = [chr(ord('A') + i) for i in range(20)]
rack_numbers = [f"{j:03d}" for j in range(1, 13)]
rack_id_list = [f"{row}{col}" for row in rack_letters for col in rack_numbers]
np.random.shuffle(rack_id_list)
item_id_prefixes = ['HK', 'KH', 'NX']
item_id_list = [f"{prefix}{str(num).zfill(3)}" for prefix in item_id_prefixes for num in range(1, 6)]
np.random.shuffle(item_id_list)
num_agents = 8
order_ids = [5001, 6001, 7001, 8001]

grid_height = 80
grid_width = 108

# 입고존의 행(Y)과 열(X) 범위
inbound_y_min = 77
inbound_y_max = 80
inbound_x_min = 80
inbound_x_max = 94

# 입고존 좌표는 이 범위 내에서 랜덤하게 생성
INBOUND_START = (np.random.randint(inbound_y_min, inbound_y_max + 1),
                 np.random.randint(inbound_x_min, inbound_x_max + 1))


# --- 직접 찍어본 출고존 좌표 반영 ---
outbound_y_min = 77
outbound_y_max = 80
outbound_x_min = 29
outbound_x_max = 43

# 출고존 좌표는 이 범위 내에서 랜덤하게 생성
OUTBOUND_GOAL = (np.random.randint(outbound_y_min, outbound_y_max + 1),
                 np.random.randint(outbound_x_min, outbound_x_max + 1))

# 랙 ID와 해당 좌표 범위를 저장할 딕셔너리
RACK_COORDS_MAP = {}

rack_letters = [chr(ord('A') + i) for i in range(20)] # 'A'부터 'T'까지
col_start_map = {}

# 랙 쌍(Pair)이 시작하는 열 번호
# A/B 쌍: 8열, C/D 쌍: 18열, E/F 쌍: 28열 ...
pair_start_col = 8

for i in range(len(rack_letters)):
    letter = rack_letters[i]
    
    # 랙 쌍마다 시작 열을 10씩 증가
    if i > 0 and i % 2 == 0:
        pair_start_col += 10
    
    # 각 쌍 내에서 첫 번째 랙은 pair_start_col, 두 번째 랙은 +2
    if i % 2 == 0:
        col_start_map[letter] = pair_start_col
    else:
        col_start_map[letter] = pair_start_col + 2

print(col_start_map)

{'A': 8, 'B': 10, 'C': 18, 'D': 20, 'E': 28, 'F': 30, 'G': 38, 'H': 40, 'I': 48, 'J': 50, 'K': 58, 'L': 60, 'M': 68, 'N': 70, 'O': 78, 'P': 80, 'Q': 88, 'R': 90, 'S': 98, 'T': 100}


In [2]:
# 랙 아이디별 행/열 좌표 범위 생성
for i in range(1, 13): # 001부터 012까지
    row_start = 5 + (i - 1) * 4
    row_end = row_start + 3
    for letter in rack_letters:
        col_start = col_start_map[letter]
        col_end = col_start + 1
        rack_id = f"{letter}{i:03d}"
        
        # 랙의 좌표 범위를 저장 (y_min, y_max, x_min, x_max)
        RACK_COORDS_MAP[rack_id] = (row_start, row_end, col_start, col_end)


# 미션 생성 루프
rows = []
# ...
for i in range(n):
    # 무작위 랙 ID를 뽑아 딕셔너리 가져옴
    rack_id = rack_id_list[i % len(rack_id_list)]
    item_id = item_id_list[i % len(item_id_list)]

    # 딕셔너리에서 랙 ID에 해당하는 좌표 범위를 가져옴
    rack_y_min, rack_y_max, rack_x_min, rack_x_max = RACK_COORDS_MAP[rack_id]
    
    # 해당 범위 내에서 무작위 좌표 생성하여 미션에 할당
    rack_y = np.random.randint(rack_y_min, rack_y_max + 1)
    rack_x = np.random.randint(rack_x_min, rack_x_max + 1)

# 랙 위치 좌표는 80x108 맵 안에서 랜덤으로 생성
RACK_COORDS = [(np.random.randint(1, grid_height + 1), np.random.randint(1, grid_width + 1)) for _ in range(n)]

rows = []
for i in range(n):
    rack_id = rack_id_list[i % len(rack_id_list)]
    item_id = item_id_list[i % len(item_id_list)]
    rack_y, rack_x = RACK_COORDS[i]

    # INBOUND 미션 (시작: 입고존, 목표: 랙)
    rows.append({
        "order_item_id": 1001 + 2*i,
        "type": "INBOUND",
        "order_id": np.random.choice(order_ids),
        "item_id": item_id,
        "requested_quantity": np.random.randint(1, 25),
        "processed_quantity": 0,
        "start_location_id": 0,
        "goal_location_id": 0,
        "start_y": INBOUND_START[0],
        "start_x": INBOUND_START[1],
        "goal_y": rack_y,
        "goal_x": rack_x,
        "rack_id": rack_id,
        "status": "WAIT",
        "priority": 1,
        "assigned_agent": ""
    })

    # OUTBOUND 미션 (시작: 랙, 목표: 출고존)
    rows.append({
        "order_item_id": 1002 + 2*i,
        "type": "OUTBOUND",
        "order_id": np.random.choice(order_ids),
        "item_id": item_id,
        "requested_quantity": np.random.randint(1, 25),
        "processed_quantity": 0,
        "start_location_id": 0,
        "goal_location_id": 0,
        "start_y": rack_y,
        "start_x": rack_x,
        "goal_y": OUTBOUND_GOAL[0],
        "goal_x": OUTBOUND_GOAL[1],
        "rack_id": rack_id,
        "status": "PENDING",
        "priority": 1,
        "assigned_agent": ""
    })

for idx, row in enumerate(rows):
    row["requested_time"] = idx + 1

columns = [
    "order_item_id", "type", "order_id", "item_id", "requested_quantity", "processed_quantity",
    "start_location_id", "goal_location_id", "start_y", "start_x", "goal_y", "goal_x",
    "rack_id", "status", "priority", "assigned_agent", "requested_time"]
df = pd.DataFrame(rows)[columns]
df.to_csv("assign_task2.csv", index=False, encoding="utf-8-sig")

In [3]:
df.head(10)

,order_item_id,type,order_id,item_id,requested_quantity,processed_quantity,start_location_id,goal_location_id,start_y,start_x,goal_y,goal_x,rack_id,status,priority,assigned_agent,requested_time
0,1001,INBOUND,5001,NX004,20,0,0,0,80,86,72,57,J001,WAIT,1,,1
1,1002,OUTBOUND,5001,NX004,2,0,0,0,72,57,79,43,J001,PENDING,1,,2
2,1003,INBOUND,7001,NX005,16,0,0,0,80,86,64,95,P003,WAIT,1,,3
3,1004,OUTBOUND,7001,NX005,2,0,0,0,64,95,79,43,P003,PENDING,1,,4
4,1005,INBOUND,8001,NX001,11,0,0,0,80,86,5,55,B012,WAIT,1,,5
5,1006,OUTBOUND,7001,NX001,9,0,0,0,5,55,79,43,B012,PENDING,1,,6
6,1007,INBOUND,7001,HK005,6,0,0,0,80,86,37,53,E002,WAIT,1,,7
7,1008,OUTBOUND,5001,HK005,11,0,0,0,37,53,79,43,E002,PENDING,1,,8
8,1009,INBOUND,8001,KH003,6,0,0,0,80,86,79,6,F006,WAIT,1,,9
9,1010,OUTBOUND,6001,KH003,3,0,0,0,79,6,79,43,F006,PENDING,1,,10
